# **Enunciado**

### Autor José Daniel Sarmiento Blanco 2192232

Design a heat exchanger to be used as a crude oil cooler. 120 kg/s of crude oil enters to the heat exchanger at 102°C and leaves at 65°C. The coolant to be used is city water entering the tube side at 21°C with a flow rate of 65 kg/s. Pressure drop for the crude oil is limited to 150 kPa.

In [21]:
caudal_crudo                = 120 # kg/s
temperatura_entrada_crudo   = 102 # ºC
temperatura_salida_crudo    = 65  # ºC

temperatura_entrada_agua    = 21  # ºC
caudal_agua                 = 65  # kg/s

presion_limite_crudo        = 150_000 # Pa

In [22]:
from pyfluids import Fluid, FluidsList, Input, Mixture
from core import ThermoFormules
import numpy as np

<p align="center">
  <img src="./diagrama.jpeg" height ="600px">
</p>

## 0. Establecer valores pre-diseño

In [23]:
k_material                            = 60    # W/ m * k
diametro_interior                     = 16e-3 # m
diametro_exterior                     = 19e-3 # m

longitud                              = 4 #m               

In [24]:
diametro_interior_c = 0.0525 # m
diametro_exterior_c = 0.0603 # m

diametro_interior_g  = 0.0779 # m

diametro_interior = diametro_interior_c

In [25]:
water = Fluid(FluidsList.Water)
water.name

Water

### 0.1 Suposición

[presión agua](https://blog.verti.es/hogar/presion-agua-casa/#:~:text=Lo%20habitual%20es%20que%20la,tengamos%20una%20presión%20u%20otra.)


- Considere como material el acero al carbono, k = 60 W/m*k.
- Diámetro de tubo de 3/4 in ( 19 mm exterior y 16 mm interior ).
- Longitud de 4m

In [26]:
presion_agua  = (2) * 101_325 # Pa
presion_crudo = 436_000       # Pa

### Composición del crudo

In [27]:
components = [FluidsList.nHexane, FluidsList.nOctane, FluidsList.nDecane]
fractions = [30, 40, 30]
mixture = dict(zip(components, fractions))
mixture

{nHexane: 30, nOctane: 40, nDecane: 30}

In [28]:
crudo = Mixture(
    components,
    fractions=fractions,
)

## 1. Termodinámica

$$ \dot{Q} = (\dot{m} *C_{p})_{crudo} * (T_{c1} -T_{c2})$$

In [29]:
temperatura_propiedades_crudo = (temperatura_entrada_crudo + temperatura_salida_crudo) / 2
print(temperatura_propiedades_crudo) #+ 273.15)

cp_propiedades_crudo =  0.83 * 1000  # J/kg k

83.5


In [30]:
calor = caudal_crudo * cp_propiedades_crudo * (temperatura_entrada_crudo - temperatura_salida_crudo ) 
calor

3685200.0

In [31]:
estado_agua_propiedades = water.with_state(
    Input.pressure(presion_agua),
    Input.temperature(34),
)
estado_agua_propiedades.as_dict()

{'compressibility': 0.0014376012709739205,
 'conductivity': 0.6203366408703609,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 994.417937160409,
 'dynamic_viscosity': 0.0007337301352239351,
 'enthalpy': 142631.9153801642,
 'entropy': 491.47808690964223,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 142428.1278251999,
 'kinematic_viscosity': 7.378488538924831e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 4.942945501085879,
 'pressure': 202650,
 'quality': None,
 'sound_speed': 1518.0155752419128,
 'specific_heat': 4179.043576032273,
 'specific_volume': 0.0010056133971753675,
 'surface_tension': None,
 'temperature': 34.0,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [32]:
cp_agua                         = estado_agua_propiedades.specific_heat # J /kg k
viscosidad_dinamica_agua        = estado_agua_propiedades.dynamic_viscosity # Pa * s
densidad_agua                   = estado_agua_propiedades.density # kg / m3
k_agua                          = estado_agua_propiedades.conductivity # W / m k
prandl_agua                     = estado_agua_propiedades.prandtl

$$ \dot{Q} = (\dot{m} *C_{p})_{agua} * (T_{w1} - T_{w2})$$

In [33]:
temperatura_salida_agua = temperatura_entrada_agua + calor / (caudal_agua * cp_agua)
print("la temperatura de lsa salida del agua del agua es: ", temperatura_salida_agua)

la temperatura de lsa salida del agua del agua es:  34.566593308704654


### 1.1 Planteamiento transferencia de calor

<p align="center">
  <img src="./resistencia.jpeg" height ="200px">
</p>

In [34]:
temperatura_propiedaes_crudo = (temperatura_entrada_crudo + temperatura_salida_crudo) / 2

estado_crudo_propiedades =  crudo.with_state(
    Input.pressure(presion_crudo),
    Input.temperature(temperatura_propiedaes_crudo),
)

densidad_crudo                  = estado_crudo_propiedades.density
viscosidad_dinamica_crudo       = estado_crudo_propiedades.dynamic_viscosity
prantl_crudo                    = estado_crudo_propiedades.prandtl
k_crudo                         = estado_crudo_propiedades.conductivity
estado_crudo_propiedades.as_dict()

{'compressibility': 0.02516409293946631,
 'conductivity': 0.12419993874048665,
 'critical_pressure': 2981833.4687300003,
 'critical_temperature': 298.33744539093857,
 'density': 642.6776111343725,
 'dynamic_viscosity': 0.00035144124174496516,
 'enthalpy': -101697.82767226541,
 'entropy': -176.77238556884234,
 'fluids': [nHexane, nOctane, nDecane],
 'fractions': [30, 40, 30],
 'freezing_temperature': None,
 'internal_energy': -102376.23935558186,
 'kinematic_viscosity': 5.468390926589865e-07,
 'max_pressure': 605929364.2590215,
 'max_temperature': 394.3155003716736,
 'min_pressure': 1.5799947781094636,
 'min_temperature': -65.24557909004545,
 'molar_mass': 0.1099927925291343,
 'phase': Liquid,
 'prandtl': 6.97825387638689,
 'pressure': 436000,
 'quality': None,
 'sound_speed': 921.561925707808,
 'specific_heat': 2466.126911171579,
 'specific_volume': 0.001555990099351567,
 'surface_tension': None,
 'temperature': 83.5,
 'triple_pressure': 1.5799947781094636,
 'triple_temperature': -65.2

In [35]:
# caudal

In [36]:
area_tubo_crudo = (np.pi / 4) * (diametro_interior_c**2)

u_m_crudo = caudal_crudo / (area_tubo_crudo * densidad_crudo)
print("la velocidad media del crudo es: ", u_m_crudo)

reynolds_crudo = 4 * caudal_crudo / (np.pi * viscosidad_dinamica_crudo * diametro_interior_c)
print("el reynolds del crudo es: ", reynolds_crudo)

f_factor_crudo = (1.58*np.log(reynolds_crudo) - 3.28)**-2
print("el factor de fricción del crudo es: ", f_factor_crudo)

nusselt_crudp = ((f_factor_crudo/2) * reynolds_crudo * prantl_crudo) \
                / (1.07 + 12.7*(f_factor_crudo**(1/2))*(prantl_crudo**(2/3)-1))

print("el nusselt del crudo es: ", nusselt_crudp)


coeficiente_convectivo_crudo = (nusselt_crudp * k_crudo) / diametro_interior_c
print("el coeficiente convectivo del crudo es: ", coeficiente_convectivo_crudo)

la velocidad media del crudo es:  86.25406805815771
el reynolds del crudo es:  8280934.252586786
el factor de fricción del crudo es:  0.0020872080594229576
el nusselt del crudo es:  23118.56226949
el coeficiente convectivo del crudo es:  54691.88605026256


In [37]:
area_tubo_agua = (np.pi / 4) * (diametro_interior_g**2 - diametro_exterior_c**2)

u_m_agua = caudal_agua / (densidad_agua * area_tubo_agua)
print("velocidad media del agua en el tubo es: ", u_m_agua)

diametro_hidraulico = diametro_interior_g - diametro_exterior_c
print("El dimetro hidraulico es: ", diametro_hidraulico)

reynolds_tubo_exterior = densidad_agua * u_m_agua * diametro_hidraulico / viscosidad_dinamica_agua
print("El reynolds del tubo exterior es: ", reynolds_tubo_exterior)

f_factor_tubo_exterior = (1.58*np.log(reynolds_tubo_exterior) - 3.28)**-2
print("El factor de fricción del tubo exterior es: ", f_factor_tubo_exterior)

nusselt_tubo_agua = ((f_factor_tubo_exterior/2) * reynolds_tubo_exterior * prandl_agua) \
                / (1.07 + 12.7*(f_factor_tubo_exterior**(1/2))*(prandl_agua**(2/3)-1))

print("El nusselt del tubo exterior es: ", nusselt_tubo_agua)

D_e = (diametro_interior_g ** 2 - diametro_exterior_c ** 2) / diametro_exterior_c

coeficiente_convectivo_agua = (nusselt_tubo_agua * k_agua) / D_e
print("El coeficiente convectivo del tubo exterior es: ", coeficiente_convectivo_agua)


velocidad media del agua en el tubo es:  34.21636065975877
El dimetro hidraulico es:  0.017599999999999998
El reynolds del tubo exterior es:  816167.0841322552
El factor de fricción del tubo exterior es:  0.0030098394599523863
El nusselt del tubo exterior es:  2534.9412947491332
El coeficiente convectivo del tubo exterior es:  38984.49757662269


Asuma los coeficiente de trasferencia de calor

$$ \frac{1}{U_{ensuciamiento}} = \frac{1}{h_{0}} + \frac{r_{0}}{r_{i}} \frac{1}{h_{i}} + \frac{r_{0}}{r_{1}}*R_{f_{}i} + R_{f_{}o} + r_{0} \frac{ln(\frac{r_{0}}{r_{i}} )}{k} $$

$$ \frac{1}{U_{limpio}} = \frac{1}{h_{0}} + \frac{r_{0}}{r_{i}} \frac{1}{h_{i}} + r_{0} \frac{ln(\frac{r_{0}}{r_{i}} )}{k} $$

In [38]:
R_f_crudo = 0.000352 # m2 k/w
R_f_agua  = 0.000176 # m2 k/w

In [41]:
U_ensuciamiento = (
    1/coeficiente_convectivo_agua + 
    R_f_crudo * diametro_exterior_c / diametro_interior_c  + 
    (diametro_exterior_c / 2) * np.log(diametro_exterior_c/diametro_interior_c) / k_material +
    (diametro_exterior_c / diametro_interior_c) * (1 / coeficiente_convectivo_crudo) +
    R_f_agua  
) ** -1
U_ensuciamiento

1435.637031304571

In [42]:
U_no_ensuciamiento = (
    1/coeficiente_convectivo_agua + 
    (diametro_exterior_c / 2) * np.log(diametro_exterior_c/diametro_interior_c) / k_material +
    (diametro_exterior_c / diametro_interior_c) * (1 / coeficiente_convectivo_crudo) 
) ** -1
U_no_ensuciamiento

8601.577022996044

In [43]:
C_f = U_ensuciamiento  / U_no_ensuciamiento
print("factor de ensuciamiento", C_f)

factor de ensuciamiento 0.1669039325540469


In [47]:
A_0 = calor / (U_ensuciamiento * (temperatura_entrada_crudo - temperatura_salida_crudo))
print("El area de transferencia de calor", A_0)

A_hp = 2 * np.pi * diametro_exterior_c * longitud
print("El area de transferencia de calor por el lado del crudo", A_hp)


relacion_area = A_0 / A_hp
print("la relacion de area es: ", relacion_area)

El area de transferencia de calor 69.37686743110335
El area de transferencia de calor por el lado del crudo 1.5155042960917162
la relacion de area es:  45.77807374747604


In [48]:
OS = 100 * U_ensuciamiento * R_f_crudo
print("porcentaje sobre la superficie", OS)

porcentaje sobre la superficie 50.534423501920905


In [49]:
caida_presion = 4 * f_factor_crudo * 2 * longitud / diametro_exterior_c *  relacion_area * densidad_crudo * u_m_crudo **2 /2 
print("la caida de presion es: ", caida_presion)

la caida de presion es:  121221114.63162172
